In [2]:
%autosave 0

Autosave disabled


In [15]:
import requests
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

In [4]:
url = "http://localhost:9696/predict"

In [8]:
house = {
        "mssubclass": 60,
        "mszoning": "RL",
        "lotfrontage": 65,
        "lotarea": 8450,
        "street": "pave",
        "alley": "na",
        "lotshape": "reg",
        "landcontour": "lvl",
        "utilities": "allpub",
        "lotconfig": "inside",
        "landslope": "gtl",
        "neighborhood": "collgcr",
        "condition1": "norm",
        "condition2": "norm",
        "bldgtype": "1fam",
        "housestyle": "2story",
        "overallqual": 7,
        "overallcond": 5,
        "yearbuilt": 2003,
        "yearremodadd": 2003,
        "roofstyle": "gable",
        "roofmatl": "compshg",
        "exterior1st": "vinylsd",
        "exterior2nd": "vinylsd",
        "masvnrtype": "brkface",
        "masvnrarea": 196,
        "exterqual": "gd",
        "extercond": "ta",
        "foundation": "pconc",
        "bsmtqual": "gd",
        "bsmtcond": "ta",
        "bsmtexposure": "no",
        "bsmtfintype1": "glq",
        "bsmtfinsf1": 706,
        "bsmtfintype2": "unf",
        "bsmtfinsf2": 0,
        "bsmtunfsf": 150,
        "totalbsmtsf": 856,
        "heating": "gasa",
        "heatingqc": "ex",
        "centralair": "y",
        "electrical": "sbrkr",
        "1stflrsf": 856,
        "2ndflrsf": 854,
        "lowqualfinsf": 0,
        "grlivarea": 1710,
        "bsmtfullbath": 1,
        "bsmthalfbath": 0,
        "fullbath": 2,
        "halfbath": 1,
        "bedroomabvgr": 3,
        "kitchenabvgr": 1,
        "kitchenqual": "gd",
        "totrmsabvgrd": 8,
        "functional": "typ",
        "fireplaces": 0,
        "fireplacequ": "na",
        "garagetype": "attchd",
        "garageyrblt": 2003,
        "garagefinish": "rfn",
        "garagecars": 2,
        "garagearea": 548,
        "garagequal": "ta",
        "garagecond": "ta",
        "paveddrive": "y",
        "wooddecksf": 0,
        "openporchsf": 61,
        "enclosedporch": 0,
        "3ssnporch": 0,
        "screenporch": 0,
        "poolarea": 0,
        "poolqc": "na",
        "fence": "na",
        "miscfeature": "na",
        "miscval": 0,
        "mosold": 2,
        "yrsold": 2008,
        "saletype": "wd",
        "salecondition": "normal"
}


In [14]:
requests.post(url, json=house).json()

{'price': 204622.625}

In [16]:
test = pd.read_csv("test.csv")
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


### cleaning the data

In [17]:
test.columns = test.columns.str.lower()

In [19]:
# translate mssubclass, OverallQual, OverallCond columns to actual names
mssubclass_values = {
    20:	"1-STORY 1946 & NEWER ALL STYLES",
    30:	"1-STORY 1945 & OLDER",
    40: "1-STORY W/FINISHED ATTIC ALL AGES",
    45:	"1-1/2 STORY - UNFINISHED ALL AGES",
    50: "1-1/2 STORY FINISHED ALL AGES",
    60:	"2-STORY 1946 & NEWER",
    70:	"2-STORY 1945 & OLDER",
    75:	"2-1/2 STORY ALL AGES",
    80:	"SPLIT OR MULTI-LEVEL",
    85:	"SPLIT FOYER",
    90:	"DUPLEX - ALL STYLES AND AGES",
    120: "1-STORY PUD (Planned Unit Development) - 1946 & NEWER",
    150: "1-1/2 STORY PUD - ALL AGES",
    160: "2-STORY PUD - 1946 & NEWER",
    180: "PUD - MULTILEVEL - INCL SPLIT LEV/FOYER",
    190: "2 FAMILY CONVERSION - ALL STYLES AND AGES",
}

test.mssubclass = test.mssubclass.map(mssubclass_values)

overallqual_values = {
       10:	"Very Excellent",
       9:	"Excellent",
       8:	"Very Good",
       7:	"Good",
       6:	"Above Average",
       5:	"Average",
       4:	"Below Average",
       3:	"Fair",
       2:	"Poor",
       1:	"Very Poor",
}
test.overallqual = test.overallqual.map(overallqual_values)

overallcond_values = {
      10:	"Very Excellent",
       9:	"Excellent",
       8:	"Very Good",
       7:	"Good",
       6:	"Above Average",
       5:	"Average",
       4:	"Below Average",
       3:	"Fair",
       2:	"Poor",
       1:	"Very Poor",
}
test.overallcond = test.overallcond.map(overallcond_values)

In [20]:
# clean the value lowercase, replace " " with "_"
for c in list(test.dtypes[test.dtypes == "object"].index):
    test[c] = test[c].str.replace(" ", "_").str.lower()

test

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition
0,1461,1-story_1946_&_newer_all_styles,rh,80.0,11622,pave,NaN,reg,lvl,allpub,...,120,0,NaN,mnprv,NaN,0,6,2010,wd,normal
1,1462,1-story_1946_&_newer_all_styles,rl,81.0,14267,pave,NaN,ir1,lvl,allpub,...,0,0,NaN,NaN,gar2,12500,6,2010,wd,normal
2,1463,2-story_1946_&_newer,rl,74.0,13830,pave,NaN,ir1,lvl,allpub,...,0,0,NaN,mnprv,NaN,0,3,2010,wd,normal
3,1464,2-story_1946_&_newer,rl,78.0,9978,pave,NaN,ir1,lvl,allpub,...,0,0,NaN,NaN,NaN,0,6,2010,wd,normal
4,1465,1-story_pud_(planned_unit_development)_-_1946_...,rl,43.0,5005,pave,NaN,ir1,hls,allpub,...,144,0,NaN,NaN,NaN,0,1,2010,wd,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,2-story_pud_-_1946_&_newer,rm,21.0,1936,pave,NaN,reg,lvl,allpub,...,0,0,NaN,NaN,NaN,0,6,2006,wd,normal
1455,2916,2-story_pud_-_1946_&_newer,rm,21.0,1894,pave,NaN,reg,lvl,allpub,...,0,0,NaN,NaN,NaN,0,4,2006,wd,abnorml
1456,2917,1-story_1946_&_newer_all_styles,rl,160.0,20000,pave,NaN,reg,lvl,allpub,...,0,0,NaN,NaN,NaN,0,9,2006,wd,abnorml
1457,2918,split_foyer,rl,62.0,10441,pave,NaN,reg,lvl,allpub,...,0,0,NaN,mnprv,shed,700,7,2006,wd,normal
